In [60]:
import pandas as pd
import os

In [ ]:
path = os.getcwd()
path_trump = path + "\\data\\hashtag_donaldtrump.csv"
trump = pd.read_csv(path_trump, lineterminator="\n")
path_biden = path + "\\data\\hashtag_joebiden.csv"
biden = pd.read_csv(path_biden, lineterminator="\n")
trump["source"] = "Trump"
biden["source"] = "Biden"
# Concatenate and remove duplicates
df = pd.concat([trump, biden], ignore_index=True)
df = df.drop_duplicates()
df = df[df["country"].isin(["United States of America", "United States"])].dropna(
    subset=["state"]
)

In [ ]:
df.isnull().sum()
print(len(df["country"]))
print(len(df["state"]))
print(len(df["city"]))

332467
332467
332467
